<img src="http://oci02.img.iteso.mx/identidad_de_instancia_2018/ITESO/Logos%20ITESO/Logo-ITESO-Principal.jpg">

<center> <font color= #004A94> <font size = 6> </font> <br> <br> <font color= #047CFB> <font size = 4>I.F. Paulina Salcedo Celis  - if708079@iteso.mx </font>

<center><font color= #004A94> <font size = 6> Microestructura y Sistemas de Trading </font> <br> <br>

<center><font color=#42c29b><font size=5> Laboratorio 1 </font> <br> <br>

<center> <font color= #047CFB> <font size = 4> Otoño 2020 </font>  |  <font color= #047CFB> <font size = 4> Repositorio: <a href='https://github.com/'>https://github.com/psc9718/myst_if708079_lab1</a></font>

# Introducción

En este laboratorio comparamos dos estrategias: Inversión Activa e Inversión Pasiva. 

Poder responder la pregunta ¿Qué estrategia de inversión propondrías si tu trabajo fuera administrar 1 Millón de pesos?

Caso: Tienes 1 Millón de pesos, tienes dos opciones para invertirlos. Por un lado, puedes hacer una Inversión Pasiva del capital, comprando el 100% del capital de títulos de un ETF que replica al mercado de renta variable mexicano S&P/BMV IPC que es el NAFTRAC. Pero por otro lado, puedes hacer una Inversión Activa del capital, al crear un portafolio que tenga los mismos activos que el ETF y haciendo el rebalanceo por medio de un algoritmo en específico.

# Objetivos
*Poder entender como es la inversión pasiva. 
*Poder entender como es la inversión activa. 
*Comparar los dos escenarios con el fin de ver cual sería la mejor opción. 

# Datos utilizados
NAFTRAC desde el 2018-2020.
Datos obtenidos de Yahoo Finance. 

# Proceso 



    Lo primero que se realizó fue la descarga y limpieza de los datos. 

In [2]:
import pandas as pd
from os import listdir, path
from os.path import isfile, join
import numpy as np
import yfinance as yf
import time as time
import functions as fn
import data as dt

ModuleNotFoundError: No module named 'yfinance'

In [3]:


### Inversión pasiva
# Lista de archivos

# Ruta absoluta de la carpeta
abspath = path.abspath('files/NAFTRAC_holdings')
archivos = [f[:-4] for f in listdir(abspath) if isfile(join(abspath, f))]

# Leer archivos
# crear un diccionario para almacenar todos los datos
data_archivos = {}

for i in archivos:
    # i = archivos[0]
    data = pd.read_csv('files/NAFTRAC_holdings/' + i + '.csv', skiprows=2, header=None)

    # Renombrara las columnas con lo que tiene el 1er renglon
    data.columns = list(data.iloc[0, :])

    # Columnas nan
    data = data.iloc[:, pd.notnull(data.columns)]

    #resetear el indice
    data = data.iloc[1:-1].reset_index(inplace=False, drop=True)
    #   quitar las comas en la columna de precios

    data['Precio'] = [i.replace(',', '') for i in data['Precio']]

    # quitar el asterisco de columna ticker
    data['Ticker'] = [i.replace('*', '') for i in data['Ticker']]

    # hacer conversiones de tipos de columnas a numerico
    convert_dict = {'Ticker': str, 'Nombre': str, 'Peso (%)': float, 'Precio': float}
    data = data.astype(convert_dict)

    # Convertir a decimal la columna de peso (%)

    data['Peso (%)'] = data['Peso (%)'] / 100

    # guardar en diccionario
    data_archivos[i] = data


Se crearon 2 funciones que sirven para las fechas y para los tickers

In [4]:
def f_fechas (archivos):

    t_fechas = [i.strftime('%d-%m-%Y') for i in sorted([pd.to_datetime(i[8:]).date() for i in archivos])]

    # lista con fechas ordenadas (para usarse como  indexadores de archivos)

    i_fechas = [j.strftime('%Y-%m-%d') for j in sorted([pd.to_datetime(i[8:]).date() for i in archivos])]

    r_f_fechas = {'i_fechas': i_fechas, 't_fechas':t_fechas}

    return r_f_fechas
# Descargar y acomodar datos

def f_ticker(archivos, data_archivos):
    tickers = []
    for i in archivos:
        # i = archivos[1]
        l_tickers = list(data_archivos[i]['Ticker'])
        print(l_tickers)
        [tickers.append(i + '.MX') for i in l_tickers]
    global_tickers = np.unique(tickers).tolist()

    # Obtener posiciones historicas


    global_tickers = [i.replace('GFREGIOO.MX', 'RA.MX') for i in global_tickers]
    global_tickers = [i.replace('MEXCHEM.MX', 'ORBIA.MX') for i in global_tickers]
    global_tickers = [i.replace('LIVEPOLC.1.MX', 'LIVEPOLC-1.MX') for i in global_tickers]

    # eliminar MXN, USD, KOFL
    [global_tickers.remove(i) for i in ['MXN.MX', 'USD.MX', 'KOFL.MX','KOFUBL.MX' ,'BSMXB.MX']]
    return global_tickers
f_ticker(archivos,data_archivos)

['AMXL', 'FEMSAUBD', 'GFNORTEO', 'WALMEX', 'GMEXICOB', 'CEMEXCPO', 'TLEVISACPO', 'ALFAA', 'GAPB', 'ASURB', 'KOFL', 'MEXCHEM', 'BIMBOA', 'AC', 'GFINBURO', 'GRUMAB', 'KIMBERA', 'BSMXB', 'IENOVA', 'PINFRA', 'PE&OLES', 'ELEKTRA', 'OMAB', 'ALSEA', 'GCARSOA1', 'MEGACPO', 'LIVEPOLC.1', 'GMXT', 'CUERVO', 'GENTERA', 'RA', 'LALAB', 'VOLARA', 'ALPEKA', 'NEMAKA', 'MXN']
['AMXL', 'FEMSAUBD', 'GFNORTEO', 'GMEXICOB', 'WALMEX', 'CEMEXCPO', 'TLEVISACPO', 'ALFAA', 'GAPB', 'ASURB', 'KOFL', 'MEXCHEM', 'BIMBOA', 'AC', 'KIMBERA', 'GFINBURO', 'IENOVA', 'GRUMAB', 'BSMXB', 'PINFRA', 'PE&OLES', 'ELEKTRA', 'OMAB', 'ALSEA', 'GCARSOA1', 'MEGACPO', 'LIVEPOLC.1', 'CUERVO', 'GENTERA', 'VOLARA', 'LALAB', 'LABB', 'GFREGIOO', 'ALPEKA', 'NEMAKA', 'MXN']
['AMXL', 'FEMSAUBD', 'GFNORTEO', 'WALMEX', 'GMEXICOB', 'CEMEXCPO', 'TLEVISACPO', 'ALFAA', 'GAPB', 'ASURB', 'MEXCHEM', 'KOFL', 'GRUMAB', 'AC', 'KIMBERA', 'IENOVA', 'BSMXB', 'BIMBOA', 'GFINBURO', 'PINFRA', 'ELEKTRA', 'OMAB', 'PE&OLES', 'ALSEA', 'GCARSOA1', 'MEGACPO', 'LIVEP

['AC.MX',
 'ALFAA.MX',
 'ALPEKA.MX',
 'ALSEA.MX',
 'AMXL.MX',
 'ASURB.MX',
 'BBAJIOO.MX',
 'BIMBOA.MX',
 'BOLSAA.MX',
 'CEMEXCPO.MX',
 'CUERVO.MX',
 'ELEKTRA.MX',
 'FEMSAUBD.MX',
 'GAPB.MX',
 'GCARSOA1.MX',
 'GCC.MX',
 'GENTERA.MX',
 'GFINBURO.MX',
 'GFNORTEO.MX',
 'RA.MX',
 'GMEXICOB.MX',
 'GMXT.MX',
 'GRUMAB.MX',
 'IENOVA.MX',
 'KIMBERA.MX',
 'LABB.MX',
 'LALAB.MX',
 'LIVEPOLC-1.MX',
 'MEGACPO.MX',
 'ORBIA.MX',
 'NEMAKA.MX',
 'OMAB.MX',
 'ORBIA.MX',
 'PE&OLES.MX',
 'PINFRA.MX',
 'Q.MX',
 'RA.MX',
 'TLEVISACPO.MX',
 'VESTA.MX',
 'VOLARA.MX',
 'WALMEX.MX']

Comenzamos por descargar los precios de los activos incluidos en el NAFTRAC

# Conclusión
